# Background

## Bayesian inference

In [ ]:
import signal
import time
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from tqdm.notebook import tqdm, trange
from abcsmc.kernels import UniformKernel, GaussianKernel, MVNKernel, LOCMKernel
from abcsmc.abcsmc import abcsmc
from abcsmc.plotting import plot_results
from abcsmc.models import GaussianModel, RingModel
from abcsmc.priors import UniformPrior, GaussianPrior

In [ ]:
np.random.seed(13242)

In [ ]:
box1 = GaussianModel(1, 1)
data = box1.generate_data(1000)
data_mu = np.mean(data)
data_var = np.var(data)

fig, ax = plt.subplots(1, 1)

ax.hist(
    data, bins=20, density=True, 
)
ax.text(0.75, 0.75, f"$\mu={data_mu:.6g}$\n$\sigma^2={data_var:.6g}$",
         transform = ax.transAxes, fontsize=10)
ax.set_xlabel("$x$")
ax.set_ylabel("Density")
ax.set_title("Observed data");

In [ ]:
prior_mu = UniformPrior(-4, 4)
prior_var = UniformPrior(0.00, 5)

def compute_loglikelihood(x, theta):
    mus = theta[:,0]
    sigmas = np.sqrt(theta[:,1])
    ll = -0.5 * ((x[:,None] - mus)/sigmas)**2 - np.log(sigmas) - 0.5*np.log(2*np.pi)
    return np.sum(ll, axis=0)

def compute_logprior(theta):
    mus = theta[:,0]
    vars = theta[:,1]
    return np.log(prior_mu.pdf(mus)) + np.log(prior_var.pdf(vars))

def compute_logposterior(theta, x):
    return compute_loglikelihood(x, theta) + compute_logprior(theta)

nparticles = 20000
params = np.zeros([nparticles, 2])
params[:,0] = prior_mu.rvs(nparticles)
params[:,1] = prior_var.rvs(nparticles)

fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(8,4))

xs = np.linspace(-10, 10, 1000)
ys = prior_mu.pdf(xs)
ax1.plot(xs, ys)

xs = np.linspace(-10, 10, 1000)
ys = prior_var.pdf(xs)
ax2.plot(xs, ys)

ax1.set_xlabel('$\mu$')
ax1.set_ylabel('PDF')
ax2.set_xlabel('$\sigma^2$')
ax2.set_ylabel('PDF')

fig.suptitle('Priors')

plt.tight_layout()

In [ ]:
ll = compute_loglikelihood(data, params)
lp = compute_logprior(params)

logposterior = ll + lp

In [ ]:
maxidx = np.argmax(logposterior)
mu_amax, var_amax = params[maxidx]

In [ ]:
gridn = 400
fig, ax = plt.subplots(1, 1)
xs = np.linspace(-5, 5, gridn)
ys = np.linspace(0.0, 6, gridn)
Xs, Ys = np.meshgrid(xs, ys)
xys = np.array([Xs, Ys]).reshape([2, -1]).T
zs = compute_logposterior(xys, data).reshape([gridn, gridn])
zs[~np.isfinite(zs)] = -np.nan

sc = ax.pcolor(xs, ys, -zs, cmap='jet_r',
    norm=LogNorm(vmin=-np.nanmax(zs), vmax=-np.nanmin(zs))
)

amax = xys[np.nanargmax(zs)]
ax.plot(
    amax[0], amax[1], 
    'ok', markersize=5,
    label= f"Max:\n$\mu={amax[0]:.6g}$\n$\sigma^2={amax[1]:.6g}$"
)

fig.colorbar(sc, ax=ax, label='$-\log$[posterior]')

ax.legend(loc='upper right')
ax.set_xlabel("$\mu$")
ax.set_ylabel("$\sigma^2$")
ax.set_title(f"Posterior: $P(\mu,\sigma^2|D)$");

In [ ]:
fig, ax = plt.subplots(1, 1)
sc = ax.scatter(
    params[:,0], params[:,1], 
    alpha=0.5,
    c=-logposterior, cmap='jet_r', s=2,
    norm=LogNorm(vmin=-logposterior.max(), vmax=-logposterior.min())
)
ax.plot(
    params[maxidx, 0], params[maxidx, 1], 
    'ok', markersize=5,
    label= f"Max:\n$\mu={mu_amax:.4g}$\n$\sigma^2={var_amax:.4g}$"
)
ax.set_xlabel("$\mu$")
ax.set_ylabel("$\sigma^2$")
ax.legend(loc='upper right')
# fig.colorbar(sc, ax=ax, label='$-\log\{p(x|\\theta)p(\\theta)\}$');
fig.colorbar(sc, ax=ax, label='$-\log$[posterior]');

In [ ]:
true_mu = 1
true_var = 1
data = GaussianModel(true_mu, true_var).generate_data(1000)
data_mu = np.mean(data)
data_var = np.var(data)

prior_mu = UniformPrior(-4, 4)
prior_var = UniformPrior(0.0, 5)
prior_list = [prior_mu, prior_var]

pname1="$\mu$"
pname2="$\sigma^2$"

def f_sim(particle, n=100):
    return GaussianModel(particle[0], particle[1]).generate_data(n)

def f_dist(x):
    mu_err = np.abs(np.mean(x) - data_mu) / data_mu
    var_err = np.abs(np.var(x) - data_var) / data_var
    return mu_err + var_err

particles, weights, results_dict = abcsmc(
    nparticles=1000, 
    nparams=2, 
    prior_list=prior_list, 
    niters=5, 
    sim_func=f_sim,
    dist_func=f_dist, 
    eps0=5, 
    eps_percentile=0.15, 
    min_eps=0, 
    kernel_method='locm'
)

particle_history = results_dict['particle_history']
weight_history = results_dict['weight_history']
score_history = results_dict['score_history']
acceptance_rates = results_dict['acceptance_rates']
epsilon_history = results_dict['epsilon_history']
niters = results_dict['niters']


In [ ]:
plot_results(
    particle_history, weight_history, score_history, acceptance_rates,
    epsilon_history, prior_list,
    pname1=pname1, pname2=pname2,
)

### Ring Example

In [ ]:
true_theta1 = 10
true_theta2 = 8
var_const = 0.5
data = RingModel(true_theta1, true_theta2, var_const).generate_data(1000)

data_mu = np.mean(data)
data_var = np.var(data)

prior_theta1 = st.uniform(-50, 100)
prior_theta2 = st.uniform(-50, 100)
prior_list = [prior_theta1, prior_theta2]

def f_sim(particle, n=100):
    return RingModel(particle[0], particle[1], var_const).generate_data(n)

def f_dist(x):
    mu_err = np.abs(np.mean(x) - data_mu) / data_mu
    var_err = np.abs(np.var(x) - data_var) / data_var
    return mu_err + var_err

particles, weights, results_dict = abcsmc(
    nparticles=1000, 
    nparams=2, 
    prior_list=prior_list, 
    niters=5,
    sim_func=f_sim,
    dist_func=f_dist, 
    eps0=5, 
    eps_percentile=0.15, 
    min_eps=0, 
    kernel_method='locm'
)

particle_history = results_dict['particle_history']
weight_history = results_dict['weight_history']
score_history = results_dict['score_history']
acceptance_rates = results_dict['acceptance_rates']
epsilon_history = results_dict['epsilon_history']


In [ ]:
plot_results(
    particle_history, weight_history, score_history, acceptance_rates,
    epsilon_history, prior_list
)

In [ ]:
true_theta1 = 5
true_theta2 = 0
var_const = 0.5
data = RingModel(true_theta1, true_theta2, var_const).generate_data(1)

prior_theta1 = st.uniform(-10, 20)
prior_theta2 = st.uniform(-10, 20)
prior_list = [prior_theta1, prior_theta2]

def f_sim(particle, n=1):
    return RingModel(particle[0], particle[1], var_const).generate_data(n)

def f_dist(x):
    return np.linalg.norm(x - data)

particles, weights, results_dict = abcsmc(
    nparticles=500, 
    nparams=2, 
    prior_list=prior_list, 
    niters=5, 
    sim_func=f_sim,
    dist_func=f_dist, 
    eps0=5, 
    eps_percentile=0.15, 
    min_eps=0, 
    kernel_method='locm',
)

particle_history = results_dict['particle_history']
weight_history = results_dict['weight_history']
score_history = results_dict['score_history']
acceptance_rates = results_dict['acceptance_rates']
epsilon_history = results_dict['epsilon_history']

In [ ]:
plot_results(
    particle_history, weight_history, score_history, acceptance_rates,
    epsilon_history, prior_list
)